**Run ED2 jobs on HPC Clusters**

This is for tonzi, harvest and umbs. There is a separate notebook for santarem job.[link text](https://)

In [ ]:
!pip install parsl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.4/466.4 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.0/264.0 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.8/224.8 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 593.7/593.7 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 37.1 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.8.0 which is incompatible.


In [ ]:
import getpass
import paramiko
import stat
import os
from parsl.app.python import PythonApp

Please provide the various input to run the job. Feel free to ignore the parameters if you wish to keep the default.

In [1]:
# Cluster details
hostname = 'cc-login.campuscluster.illinois.edu' # the hostname of the cluster you want to run the ed2 model
password = None
username = "ABCD" # username on the cluster


# Batch job details
time = "00:15:00" # Job run time (hh:mm:ss)
nodes = 1 # Number of nodes
ntasks_per_node = 16             # Number of task (cores/ppn) per node
job_name = "ED2IN-umbs.bg"           # Name of batch job
partition = "secondary"            # Partition (queue)
output = "openmp_umbs.o%j"              # Name of batch job output file
error = "openmp_umbs.e%j"               # Name of batch job error file
mail_user = "NetID@illinois.edu"  # Send email notifications
mail_type = "BEGIN,END"           # Type of email notifications to send


# Path to various inputs
path_to_data = "${HOME}/ed-demo" #path to data for ED2 on cluster
path_to_singularity_image = "${HOME}/ed2-intel.sif" # path to singularity image of ED2 model on cluster
path_to_ED2IN_file = "ED2IN-umbs.bg" # path to ED2IN file on cluster, relative path to path_to_data is okay

In [18]:
@PythonApp
def run_singularity(executable, singularity_image, args, stdout=None, stderr=None):
    return f"{executable} exec {singularity_image} {args}"

def submit_job(username):
    ssh_client = paramiko.SSHClient()
    ssh_client.load_system_host_keys()
    ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    try:
        ssh_client.connect(hostname, username=username, password=password, allow_agent=True)
        print("successfully connected")
    except:
        pass
    transport = ssh_client.get_transport()
    transport.auth_password(username, getpass.getpass('Enter {0} Logon password :'.format(hostname)))
    sftp_client = paramiko.SFTPClient.from_transport(transport)

    # Run the .bat file which is alreday present on cluster
    #_, stdo, stde = ssh_client.exec_command(command_line)
    #print(stdo.read().decode())
    #print(stde.read().decode())


    #create the bat file
    with open(job_name + ".sbatch", 'w') as f:
        f.writelines("#!/bin/bash\n")
        f.writelines("#SBATCH --time=" + str(time) + "\n")
        f.writelines("#SBATCH --ntasks-per-node=" + str(ntasks_per_node) + "\n")
        f.writelines("#SBATCH --job-name=" + job_name + "\n")
        f.writelines("#SBATCH --partition=" + partition + "\n")
        f.writelines("#SBATCH --output=" + output + "\n")
        f.writelines("#SBATCH --error=" + error + "\n")
        f.writelines("#SBATCH --mail-user=" + mail_user + "\n")
        f.writelines("#SBATCH --mail-type=" + mail_type + "\n")
        f.writelines("\n")
        f.writelines("module load singularity" + "\n")
        f.writelines("singularity exec --bind " + path_to_data + ":/data --no-home --pwd /data " + path_to_singularity_image + " ed2 -f " + path_to_ED2IN_file)
    f.close()

    #transfer .bat file to cluster and run it
    sftp_client.put(job_name + ".sbatch", f"/home/{username}/" + job_name + ".sbatch")
    sftp_client.chmod(f"/home/{username}/" + job_name + ".sbatch", stat.S_IRWXU)
    _, stdo, stde = ssh_client.exec_command("sbatch " + job_name + ".sbatch")
    print(stdo.read().decode())
    print(stde.read().decode())

    sftp_client.close()
    ssh_client.close()
    transport.close()

submit_job(username)

Enter cc-login.campuscluster.illinois.edu Logon password :··········
Submitted batch job 9829951


